<h1>Stations table</h1>

<h3>IMPORTS</h3>

In [1]:
#IMPORTS
import pandas as pd
import os
from google.cloud.sql.connector import Connector, IPTypes
import pymysql
import sqlalchemy
from zipfile import ZipFile
from datetime import datetime, timedelta

<h3>METHODS</h3>

In [142]:
def extract_stations_data():
    filepath = "./../Data/stations_db.csv"

    # Read raw file as lines
    with open(filepath, "r") as f:
        lines = f.readlines()

    rows = []
    for line in lines:
        parts = line.strip().split()
        
        # First 3 are fixed
        first3 = parts[:3]
        
        # Last 2 are always lat/lon (sometimes lon missing)
        # Last 3rd is always the date
        last3 = parts[-3:]
        
        # Everything in between belongs to the "Name"
        name = " ".join(parts[3:-3])
        
        row = first3 + [name] + last3
        rows.append(row)

    # Create DataFrame
    df = pd.DataFrame(rows, columns=["station_id", "State", "Code", "Station Name", "Date", "Lat", "Lon"])
    df = df[['station_id', "Station Name", "State", "Code", "Lat", "Lon"]]

    return df


extract_stations_data()


,station_id,Station Name,State,Code,Lat,Lon
0,015590,ALICE SPRINGS AIRPORT,NT,15B,-23.7951,133.889
1,014272,BATCHELOR AIRPORT,NT,14GA,-13.0544,131.0252
2,014723,BORROLOOLA AIRPORT,NT,14DE,-16.0755,136.3041
3,014954,BRADSHAW,NT,14F,-14.9408,130.8091
4,014808,BRADSHAW - ANGALLARI VALLEY (DEFENCE),NT,14F,-15.4397,130.5731
...,...,...,...,...,...,...
509,013011,WARBURTON AIRFIELD,WA,13,-26.1317,126.5839
510,013044,WILUNA AERO,WA,13,-26.6273,120.2195
511,109521,WITCHCLIFFE WEST,WA,09A,-34.0258,115.0637
512,001006,WYNDHAM AERO,WA,01,-15.51,128.1503


<h3>Establish file paths & connections</h3>

In [134]:
#Connection to database
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of MySQL.

    Uses the Cloud SQL Python Connector package.
    """

    instance_connection_name = "esoteric-life-470902-k9:australia-southeast2:jyjam-weatherdata-db"
    db_user = "admin" 
    db_pass = "Password1!"
    db_name ="weather_data_database"

    ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector(ip_type=ip_type, refresh_strategy="LAZY")

    def getconn() -> pymysql.connections.Connection:
        conn: pymysql.connections.Connection = connector.connect(
            instance_connection_name,
            "pymysql",
            user=db_user,
            password=db_pass,
            db=db_name,
        )
        return conn

    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )
    return pool


In [136]:
#connection to database
db_conn = connect_with_connector()
db_table = 'stations'

<h3>CREATE DATAFRAME</h3>

In [144]:
stations_df = extract_stations_data()

<h3>RESET TABLE ROWS</h3>

In [ ]:
cursor = db_conn.cursor()


sql_create_unique_table = "CREATE TABLE dates_temp AS SELECT DISTINCT * FROM Dates"
sql_drop_old_table = "DROP TABLE Dates"
sql_create_new_table = "CREATE TABLE Dates AS SELECT DISTINCT * FROM dates_temp"
sql_drop_temp_table = "DROP TABLE dates_temp"

try:
    # Execute the DELETE statement
    cursor.execute(sql_create_unique_table)
    cursor.execute(sql_drop_old_table)
    cursor.execute(sql_create_new_table)
    cursor.execute(sql_drop_temp_table)


    # Commit the changes to the database
    db_conn.commit()

    print(f"Distinct '{db_table}' table has been created.")

except sqlite3.Error as e:
    print(f"Error creating distinct table: {e}")

finally:
    # Close the cursor
    cursor.close()

Distinct 'Dates' table has been created.


<h3>INSERT ALL Data into weather_data table</h3>

In [148]:
with db_conn.connect() as cursor:
    stations_df.to_sql(db_table, cursor, if_exists='append', index=False)